In [1]:
%matplotlib inline

from datetime import datetime

import datacube
import utils.data_cube_utilities.dc_utilities as utils

from utils.data_cube_utilities.dc_mosaic import create_mosaic, ls7_unpack_qa
from utils.data_cube_utilities.dc_water_classifier import *

import warnings; warnings.simplefilter('ignore')

In [2]:
start_time = datetime.now()
print("Start time: " + str(start_time))

Start time: 2018-04-20 10:49:24.203585


In [3]:
# Initialize data cube object
dc = datacube.Datacube(config='/home/localuser/.datacube.conf', app='dc-coastal-erosion')

# Set query parameters
platform        = 'LANDSAT_7'
product_type    = 'ls7_ledaps_togo'

min_lon         = 1.1919
max_lon         = 1.4509
min_lat         = 6.0809
max_lat         = 6.2218

In [4]:
range(2000, 2015)

range(2000, 2015)

In [5]:
start_date      = datetime.strptime('2007-01-01', '%Y-%m-%d')
end_date        = datetime.strptime('2007-12-31', '%Y-%m-%d')

In [6]:
measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa']
# Retrieve data from Data Cube
dataset_in = dc.load(platform=platform,
                     product=product_type,
                     time=(start_date, end_date),
                     lon=(min_lon, max_lon),
                     lat=(min_lat, max_lat), measurements=measurements)

In [7]:
# clean_mask = utils.create_cfmask_clean_mask(dataset_in.cf_mask)

clear_xarray  = ls7_unpack_qa(dataset_in.pixel_qa, "clear")  
water_xarray  = ls7_unpack_qa(dataset_in.pixel_qa, "water")
clean_mask = np.logical_or(clear_xarray.values.astype(bool),
                           water_xarray.values.astype(bool))
mosaic = create_mosaic(dataset_in, clean_mask=clean_mask)

In [8]:
mosaic

<xarray.Dataset>
Dimensions:    (latitude: 520, longitude: 957)
Coordinates:
  * latitude   (latitude) float64 6.222 6.222 6.221 6.221 6.221 6.22 6.22 ...
  * longitude  (longitude) float64 1.192 1.192 1.192 1.193 1.193 1.193 1.193 ...
Data variables:
    red        (latitude, longitude) int16 1797 1797 1797 1797 1824 1769 ...
    green      (latitude, longitude) int16 1613 1643 1613 1613 1643 1613 ...
    blue       (latitude, longitude) int16 1410 1438 1410 1410 1410 1438 ...
    nir        (latitude, longitude) int16 2231 2231 2231 2231 2231 2272 ...
    swir1      (latitude, longitude) int16 2772 2623 2586 2586 2623 2586 ...
    swir2      (latitude, longitude) int16 2420 2227 2304 2227 2420 2227 ...
    pixel_qa   (latitude, longitude) int32 66 66 66 66 130 130 130 130 66 66 ...

In [9]:
water_class = wofs_classify(mosaic, mosaic=True, clean_mask=
                            np.ones(mosaic.red.shape).astype(np.bool))
water_class.wofs.values[water_class.wofs.values == -9999] = np.nan
wofs = water_class.wofs.values

In [10]:
wofs[mosaic.swir2.values > 100] = np.nan

In [11]:
coastline = np.zeros(wofs.shape)

for i in range(wofs.shape[0]):
    for j in range(wofs.shape[1]):
        pixel = wofs[i,j]
        if pixel == 0 and np.nansum(wofs[i-1:i+2, j-1:j+2]) >= 1 and np.nansum(wofs[i-1:i+2, j-1:j+2]) <= 5:
            coastline[i,j] = 1

In [12]:
len(water_class.latitude)

520

In [13]:
wofs[np.isnan(wofs)] = -9999

In [14]:
wofs

array([[-9.999e+03, -9.999e+03, -9.999e+03, ..., -9.999e+03, -9.999e+03,
        -9.999e+03],
       [-9.999e+03, -9.999e+03, -9.999e+03, ..., -9.999e+03, -9.999e+03,
        -9.999e+03],
       [-9.999e+03, -9.999e+03, -9.999e+03, ..., -9.999e+03, -9.999e+03,
        -9.999e+03],
       ...,
       [-9.999e+03, -9.999e+03, -9.999e+03, ..., -9.999e+03, -9.999e+03,
        -9.999e+03],
       [-9.999e+03, -9.999e+03, -9.999e+03, ..., -9.999e+03, -9.999e+03,
         1.000e+00],
       [-9.999e+03, -9.999e+03, -9.999e+03, ..., -9.999e+03, -9.999e+03,
        -9.999e+03]])

In [15]:
driver = gdal.GetDriverByName('GTiff')
raster = driver.Create('2007_wofs.tif', len(water_class.longitude), len(water_class.latitude), 1, gdal.GDT_Float32)

crs = dataset_in.crs
spatial_ref = utilities.get_spatial_ref(crs)

# Upper left coordinates
ul_lon = dataset_in.longitude.values[0]
ul_lat = dataset_in.latitude.values[0]

# Resolution
products = dc.list_products()
resolution = products.resolution[products.name == 'ls7_ledaps_togo']
lon_dist = resolution.values[0][1]
lat_dist = resolution.values[0][0]

# Rotation
lon_rtn = 0
lat_rtn = 0

geotransform = (ul_lon, lon_dist, lon_rtn, ul_lat, lat_rtn, lat_dist)

raster.SetGeoTransform(geotransform)
raster.SetProjection(spatial_ref)

out_band = raster.GetRasterBand(1)
out_band.SetNoDataValue(-9999)
out_band.WriteArray(wofs)
out_band.FlushCache()

raster.FlushCache()
out_band = None
raster = None

In [16]:
end_time = datetime.now()
print("Elapsed time: " + str(end_time - start_time))

Elapsed time: 0:00:04.527022
